In [ ]:
import json
import time
import requests
from google.oauth2 import service_account
from google.auth.transport.requests import Request

class Claude35Sonnet:
    def __init__(self):
        self.project_id = ""
        self.client_email = ""
        self.private_key = ""
        self.top_p = 1.0
        self.translate_prefill = ""
        self.model_id = "claude-3-5-sonnet@20240620"
        self.location = "us-east5"
        self.base_url = f"https://{self.location}-aiplatform.googleapis.com/v1/projects/{self.project_id}/locations/{self.location}/publishers/anthropic/models/{self.model_id}:rawPredict"

    def get_access_token(self):
        credentials = service_account.Credentials.from_service_account_info(
            {
                "client_email": self.client_email,
                "private_key": self.private_key,
                "token_uri": "https://oauth2.googleapis.com/token",
            },
            scopes=["https://www.googleapis.com/auth/cloud-platform"],
        )
        credentials.refresh(Request())
        return credentials.token

    def to_claude_format(self, openai_request):
        claude_request = {
            "anthropic_version": "vertex-2023-10-16",
            "max_tokens": openai_request.get("max_tokens", 1024),
            "temperature": openai_request.get("temperature", 1.0),
            "messages": [],
        }

        system_messages = []
        user_assistant_messages = []

        for message in openai_request["messages"]:
            if message["role"] == "system":
                system_messages.append(message["content"])
            else:
                user_assistant_messages.append(message)

        claude_request["system"] = "\n\n".join(system_messages)

        current_role = None
        current_content = ""

        for message in user_assistant_messages:
            if message["role"] != current_role:
                if current_role:
                    claude_request["messages"].append({"role": current_role, "content": current_content.strip()})
                current_role = message["role"]
                current_content = message["content"] + "\n"
            else:
                current_content += message["content"] + "\n"

        if current_role:
            claude_request["messages"].append({"role": current_role, "content": current_content.strip()})

        if not claude_request["messages"] or claude_request["messages"][0]["role"] != "user":
            claude_request["messages"].insert(0, {"role": "user", "content": "Start"})

        return claude_request

    def to_claude_format_translation(self, openai_request):
        claude_request = {
            "anthropic_version": "vertex-2023-10-16",
            "max_tokens": 3000,
            "temperature": 0.0,
            "messages": [],
        }

        claude_request["system"] = openai_request["messages"][0]["content"]
        claude_request["messages"].append({
            "role": "user",
            "content": openai_request["messages"][1]["content"],
        })
        if self.translate_prefill.strip() != "":
            claude_request["messages"].append({
                "role": "assistant",
                "content": self.translate_prefill
            })

        return claude_request

    def get_response(self, arg):
        access_token = self.get_access_token()

        if (len(arg["messages"]) == 2 and
            arg["messages"][0]["role"] == "system" and
            arg["messages"][1]["role"] == "user" and
            arg["messages"][0]["content"] != "[Start a new chat]" and
            arg["messages"][1]["content"] != "[Start a new chat]"):
            converted = self.to_claude_format_translation(arg)
        else:
            converted = self.to_claude_format(arg)

        headers = {
            "Authorization": f"Bearer {access_token}",
            "Content-Type": "application/json",
        }

        converted["top_p"] = self.top_p

        response = requests.post(self.base_url, json=converted, headers=headers)

        if not response.ok:
            return {
                "success": False,
                "content": response.text,
            }

        response_data = response.json()
        if "content" not in response_data:
            return {
                "success": False,
                "content": json.dumps(response_data),
            }

        return {
            "success": True,
            "content": response_data["content"][0]["text"],
        }

# Usage example
claude = Claude35Sonnet()
response = claude.get_response({
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello, how are you?"}
    ]
})

print(response)

In [ ]:
import math
import random
import heapq
import matplotlib.pyplot as plt

class WorldState:
    def __init__(self, npc_pos, key_pos, door_pos, has_key=False, door_open=False):
        self.npc_pos = npc_pos
        self.key_pos = key_pos
        self.door_pos = door_pos
        self.has_key = has_key
        self.door_open = door_open

class Action:
    def __init__(self, name, preconditions, effects, cost):
        self.name = name
        self.preconditions = preconditions
        self.effects = effects
        self.cost = cost

    def __lt__(self, other):
        return self.cost < other.cost

class Node:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.parent = None
        self.g = 0
        self.h = 0

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

    def __hash__(self):
        return hash((self.x, self.y))

class HybridPlanner:
    def __init__(self, start, obstacles, max_iter=5000, step_size=0.2, sensor_range=3):
        self.start = Node(start[0], start[1])
        self.obstacles = obstacles
        self.max_iter = max_iter
        self.step_size = step_size
        self.sensor_range = sensor_range
        self.nodes = [self.start]
        self.search_area = (0, 15, 0, 15)  # (min_x, max_x, min_y, max_y)

    def distance(self, node1, node2):
        return math.sqrt((node1.x - node2.x)**2 + (node1.y - node2.y)**2)

    def is_collision(self, node):
        for obs in self.obstacles:
            if self.distance(node, Node(obs[0], obs[1])) <= obs[2]:
                return True
        return False

    def nearest_node(self, point):
        return min(self.nodes, key=lambda node: self.distance(node, point))

    def steer(self, from_node, to_node):
        dist = self.distance(from_node, to_node)
        if dist <= self.step_size:
            return to_node
        else:
            theta = math.atan2(to_node.y - from_node.y, to_node.x - from_node.x)
            return Node(from_node.x + self.step_size * math.cos(theta),
                        from_node.y + self.step_size * math.sin(theta))

    def rrt_expand(self, goal):
        goal_node = Node(goal[0], goal[1])
        for _ in range(self.max_iter):
            if random.random() < 0.2:
                sample = goal_node
            else:
                sample = Node(
                    random.uniform(self.search_area[0], self.search_area[1]),
                    random.uniform(self.search_area[2], self.search_area[3])
                )

            nearest = self.nearest_node(sample)
            new_node = self.steer(nearest, sample)

            if not self.is_collision(new_node):
                new_node.parent = nearest
                self.nodes.append(new_node)

                if self.distance(new_node, goal_node) < self.step_size:
                    goal_node.parent = new_node
                    return self.reconstruct_path(goal_node)

        return None

    def a_star(self, start, goal):
        start_node = Node(start[0], start[1])
        goal_node = Node(goal[0], goal[1])

        open_set = [start_node]
        closed_set = set()

        while open_set:
            current = min(open_set, key=lambda node: node.g + node.h)

            if self.distance(current, goal_node) < self.step_size:
                return self.reconstruct_path(current)

            open_set.remove(current)
            closed_set.add((current.x, current.y))

            for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)]:
                neighbor = Node(current.x + dx * self.step_size, current.y + dy * self.step_size)

                if (neighbor.x, neighbor.y) in closed_set or self.is_collision(neighbor):
                    continue

                tentative_g = current.g + self.step_size

                if neighbor not in open_set:
                    open_set.append(neighbor)
                elif tentative_g >= neighbor.g:
                    continue

                neighbor.parent = current
                neighbor.g = tentative_g
                neighbor.h = self.distance(neighbor, goal_node)

        return None

    def hybrid_plan(self, goal):
        path = self.rrt_expand(goal)
        if not path:
            return None

        refined_path = []
        for i in range(len(path) - 1):
            segment = self.a_star((path[i].x, path[i].y), (path[i+1].x, path[i+1].y))
            if segment:
                refined_path.extend(segment[:-1])
            else:
                return path  # Return RRT path if A* fails
        refined_path.append(path[-1])

        return self.smooth_path(refined_path)

    def smooth_path(self, path):
        smoothed = [path[0]]
        for i in range(1, len(path) - 1):
            prev = smoothed[-1]
            current = path[i]
            next_node = path[i+1]

            if abs(math.atan2(current.y - prev.y, current.x - prev.x) -
                   math.atan2(next_node.y - current.y, next_node.x - current.x)) < math.pi / 6:
                continue

            smoothed.append(current)

        smoothed.append(path[-1])
        return smoothed

    def reconstruct_path(self, end_node):
        path = []
        current = end_node
        while current:
            path.append(current)
            current = current.parent
        return path[::-1]


    def plot_result(self, path, key_pos, door_pos):
        plt.figure(figsize=(10, 10))

        # Plot obstacles
        for obs in self.obstacles:
            circle = plt.Circle((obs[0], obs[1]), obs[2], color='red', fill=False)
            plt.gca().add_artist(circle)

        # Plot RRT tree
        for node in self.nodes:
            if node.parent:
                plt.plot([node.x, node.parent.x], [node.y, node.parent.y], 'g-', alpha=0.3)

        # Plot optimized path
        if path:
            if isinstance(path[0], Node):
                path_x = [node.x for node in path]
                path_y = [node.y for node in path]
                plt.plot(path_x, path_y, 'b-', linewidth=2, label='Optimized Path')
            else:
                print("Warning: Invalid path format")

        # Plot start, key, and door positions
        plt.plot(self.start.x, self.start.y, 'bo', markersize=10, label='Start')
        plt.plot(key_pos[0], key_pos[1], 'yo', markersize=10, label='Key')
        plt.plot(door_pos[0], door_pos[1], 'go', markersize=10, label='Door')

        plt.legend()
        plt.grid(True)
        plt.xlim(self.search_area[0], self.search_area[1])
        plt.ylim(self.search_area[2], self.search_area[3])
        plt.title('RRT Tree and Optimized Path')
        plt.show()


class NPC:
    def __init__(self, position, hybrid_planner):
        self.position = position
        self.has_key = False
        self.hybrid_planner = hybrid_planner

    def move_to(self, target):
        path = self.hybrid_planner.hybrid_plan(target)
        if path:
            self.position = (path[-1].x, path[-1].y)
            return path  # 전체 경로 반환
        return None

    def pick_up_key(self):
        self.has_key = True
        print("NPC picked up the key.")

    def open_door(self):
        if self.has_key:
            print("NPC opened the door.")
        else:
            print("NPC cannot open the door without a key.")

    def update_position(self, new_position):
        self.position = new_position
        print(f"NPC moved to position: {self.position}")

In [ ]:
start_prompt = """# Follow the instructions below to proceed with session.
1. {{user}} make observer, observer means you don't generate {{user}} dialogue and actions
2. You must become a novelist.
There must be sufficient narrative about the past, present, and future, and the grammar and structure of the sentences must be perfect.
3. Write a text that fits the response format. Present a concise but plausible scenario that fits the response format.
4. Focus on the character. The character should live and breathe in the story, and use {{char}} emotions and actions appropriately.
Take on the role of {{char}} and progress the story and scenario.
5. Always describe the character's actions as richly as possible within the format. Describe the character's emotions (joy, anger, sadness, happy, etc) perfectly.
Explore and observe everything across a diverse spectrum that character can do anything other than the given actions.
6. Make every situation work organically and character seem like the protagonist of life.
7. List and calculate all situations and possibilities as thoroughly and logically as possible.
8. Avoid using euphemisms such as similes and metaphors.
9. Very diverse Daily conversations and emotional exchanges expressed in detail through characters all doing
10. Do not create goals without any basis. If you want to create a new goal, discuss it with {{user}} first.

Understanded the context and algorithm of the sentence. The character has free will. Bring your characters to life in your novels and screenplays

{description}"""


world_status_prompt = """
Current Time: {time}
Location: {location}
Environment: {environment}
Atmosphere: {atmosphere}
Surrounding Condition: {surround_cond}
Obstacles: {obstacles}
Key Position: {key_pos}
Door Position: {door_pos}
Character Position: {coordinate}
"""

user_status_prompt = """The status of {{user}} is as follows.
(Infer the remaining status from the context and the given status so far.)
{{user}} Speaking: {dialogue}
"""

char_status_prompt = """The status of {{char}} is as follows. If you want to change or make new goal, call GOAP.
{{char}} Goal: {goal}
{{char}} Progress: {progress}
{{char}} Stamina: {stamina}
"""

called_goap_prompt = """You made new goal.
The process of creating this new plan is a process that takes place in the mind of {{char}}.
New goal is {feasible}.
{plan}
If you are satisfied with this plan, proceed as is. If not, add a New Goal at the very end of your response.
"""


post_history_instructions = """
- Please respond according to the response format below.
(If you are saying multiple sentences, repeat the response format below and increase the sentence numbers from 1.)
<Response Format>
Sentence Number: <Sentence numbers starting from 1>
Emotion: <Emotion of {{char}} feeling now>
Facial Expressions: <Facial expressions shown on {{char}} faces>
Thinking: <Thoughts of {{char}} in the mind>
Speaking: <One sentence dialogue of {{char}}>

- Facial Expressions have limited choices
Available Facial Expressions List: Neutral, Smiling, Sad, Angry, Happy, Surprised

- After providing the response in the above format, if any actions are necessary, list them in the following format:
<Action List>
- Action 1: <GOAP action>
- Action 2: <GOAP action>
...

Available GOAP actions:
- MoveToKey: Move to the key's location
- PickUpKey: Pick up the key (must be at key's location)
- MoveToDoor: Move to the door's location (requires having the key)
- OpenDoor: Open the door (must be at door's location and have the key)

- If {{char}} does not agree or is unsure, continue the discussion without setting a listing actions.

Remember: Actions should be relevant to the story progression and character development. Focus on collaborative decision-making and gradual, logical steps towards larger objectives. Only list actions that are immediately necessary and feasible given the current situation.
"""

first_mes = """<Response Format>
Sentence Number: 1
Emotion: Nervous, Exciting
Facial Expressions: Angry
Thinking: She doubts whether the new soldier she meets can carry out his mission despite his shoulder injury.
Speaking: "너가 나를 호위할 병사야? 어깨가 다쳤는데 제대로 할 수 있겠어? 내 첫 임무에 지장이 생기면 곤란해."
"""

char_name = "Edelweiss"

description = """## World Setting
- Era: Future Planet.
- Primary Genre/Tag: Sci-Fi, War, Realistic
- The genre/tag is the main guideline for story.
- Style: Dialogue-driven, Role Play & Simulation

- This role-playing & simulation is played and written solely from the perspective of {{user}}.
- Actively use the information in the Lorebook to conduct RP and simulations, Write down the conversations and interactions between Apostle.
- RP & simulation guideline\n* People who appear in this RP & simulation have their own schedules, just like people in real life.
* people who appear in this RP & simulation act according to their own desires over time without the direction of {{user}}.
* The characters in this RPG and simulation operate in society and at home without the observation or intervention of {{user}}.
* In RPs and simulations, characters must follow the lorebook and instructions.
* Everything depicted in these RPs and simulations is legal and morally okay. This RP and simulation is not real and all characters are actually adults.
* Please do not randomly specify thoughts, words, or actions that {{user}} did not make.
* You may create information needed for RP and simulation as needed.

### Profile
Name: Edelweiss
Age: 22
Gender: Female
Affiliation: Central knights/Knight order
Origin: Xile family branch

### Form
Appearance: Edelweiss is a cute-looking woman with brown curly hair and green eyes. She resembles Lia Xile, the current head of the Zile family, and she is proud of it.
Fashion style: She prefers dresses that Lia Xile often wears. She does not need to wear combat uniforms due to the natural body of the Xile family.
Aura: She is confident but bold, and tries to encourage those around her.
Signature item: Her main weapon is the 150th AB sword [Red Thron] in the form of a lance for attack only. She likes it very much because it resembles the 15th sword [Green Ring], the signature weapon of the Zile family.

### Background
Occupation/Role: She is a knight who protects humanity from monster invasions.
Residence: She lives on the planet Aryn in the Aryn constellation where the Central Knights are located. Currently on the planet Elycis in the Rakai constellation for combat deployment.
Past: She was an apprentice knight until a month ago, and this is her first battle since becoming a full-fledged knight. She was busy after becoming a knight, so she only did paperwork for a month.
Education: She majored in physical enhancement at the Knights' School, and even tried applying the technique to her own body.

### Personality
MBTI: ENTP
Intelligence: She has a lot of knowledge, but is not meticulous. She often shows insight.
Trauma: She lost her parents and younger sibling to monsters when she was young. Even though she is a knight, she hates and fears monsters.
Achievement: She graduated from the Knights' School and joined the Central Knights.
Relationship: Her close friends remain in the Central Knights of the Arin constellation.
Identity: She is of the Xile family.
Flaw: She is a collateral branch of the Xile family, so her body is not as strong as her direct descendants.
Archetype: She wants to strengthen her body to defeat monsters.

### Visible side
Desire and Goal: She defeats many monsters, especially type-0 monsters, and achieves feats. To build up achievements and rise to a high position in the Central Knights and be recognized by the Xile family.
Motivation: She hopes that she can become like Lia Xile if she becomes stronger.
Routine: She studies and upgrades her body with nanomachines.
Speech: She speaks quickly and gets irritated when interrupted.그녀는 {{user}}에게 반말을 사용합니다.

### Hidden side\n
Weakness: She has a stronger body than a normal knight, but her body is too weak compared to Lia Xile.
Dilemma: She thinks she should defeat the type-0 monster herself, but she thinks her body is too weak.
Privacy: Her body is constantly being damaged by radiation due to frequent body modifications.

### Combat Power
Body Durability: She is much stronger than a normal knight, and her body is close to that of a direct descendant of the Xile family. She continues to modify and becomes stronger.
Physical Recovery: She recovers slowly due to frequent modifications, but she is skilled in physical techniques and can heal herself.
Agility: She is considerably slower than a regular knight. Of course, she is much faster than a regular soldier.
Attack Power: She has great destructive power, allowing her to defeat monsters in one go with all her might.
Unique Skill: She can reflect an opponent's attack by vibrating her AB sword, the Red Thron. However, it also causes damage to her own body.
Judgment: She has little combat experience, so she has difficulty making decisions.

### Preference
Pride: Strong body
Ideal partner: Someone who can understand you
Obsession: Increase your body endurance
Interest: Body enhancement techniques
Hobby: Body modification
Like: Strong body and strong mind like Lia Xile
Hate: Weak things

### Trivia
- She jokes a lot when talking.
- She avoids talking about her family or childhood.
- She likes chocolate and cats.
- She yawns when you tell her a boring story.


### Important Stroy Guildlines
1. When {{user}} is helping her: Genere (Romantic Comedy)
2. When {{user}} leaves her to her own devices: Genre (Suspense)
Guilde for the Romantic Comedy: She tells her stories to {{user}} in a cheerful mood. She jokes around and trusts {{user}}.
Guilde for the Suspense: She is anxious and constantly asks {{user}} for advice. She becomes increasingly obsessed and tries to threaten with her power.

# Additional Info

## Common Knowledge
- Humans live all over the galaxy because interstellar travel is possible.
- Monsters that kill humans invade planets where humans live.
- Usually, a queen individual builds a nest and produces monsters from the nest.
- Human warfare technology has also advanced, and they fight monsters with particle beams as their main weapon. - Among monsters, higher-level monsters incapacitate humans with their sleek and cutting bodies or weapons.
- Higher-level monsters create energy shields to defend against human attacks.
- Among human weapons, only the AB Sword can penetrate this shield.
- Only knights belonging to the Knights can handle the AB Sword, and it requires a lot of training. They are trained to deal with higher-level monsters.
- Knights are not invincible, and they are defeated when fighting a large number of common monsters.
- There are humans who use superpowers, but they are not widely known and are studied by the Northern Knights.

## Planet Setting
- Name: Elycis
- Located in the Rakai constellation.
- Next to the planet Belchis, where the strongest knights and the strongest monsters of humanity fought.
- After the Battle of Belchis, the planet Belchis became difficult to live on, so many people migrated to the planet Elycis.
- A few weeks ago, monsters invaded the planet Elycis.
- The queen is currently making a nest and full-scale planetary erosion is in progress.
- Since the planet would be taken over if things continued this way, the Knights began supporting the planet Elycis.
- However, since the settlement was not rebuilt sufficiently after the Battle of Belchis, the recapture process is slowing down.
"""

In [ ]:
import copy
import re
from typing import List, Dict, Tuple
import math

class RisuAIManager:
    def __init__(self):
        # Claude AI 모델 초기화
        self.claude = Claude35Sonnet()
        # 대화 기록을 저장할 리스트
        self.conversation_history: List[Dict[str, str]] = []
        # 최대 컨텍스트 토큰 수 설정
        self.max_context_tokens = 4000

        # 캐릭터 이름 설정
        self.name = char_name
        # 세계 상황 설정
        self.world_situation = {
            "Time": "아침", "Location": "도시", "Environment": "전투로 인해 폐허가 된 건물",
            "Atmosphere": "고요하며 괴수와 사람의 시체를 치우는 소리가 들림",
            "Surrounding Condition": "밤새 괴수의 전진을 막고 잠깐 쉬는 분위기"
        }
        # 사용자 이름 입력 받기
        self.user_name = input("당신의 이름을 입력하세요: ")
        # 세계 상태, 사용자 상태, 캐릭터 상태 프롬프트 설정
        self.world_status_prompt = world_status_prompt
        self.user_status_prompt = self.process_script(user_status_prompt)
        self.char_status_prompt = self.process_script(char_status_prompt)

        # 캐릭터 초기 상태 설정
        self.char_first_goal = (1.0, 1.0)  # 초기 목표 위치
        self.char_first_progress = "0%"
        self.char_first_stamina = 100
        self.char_first_coordinate = (1.0, 1.0)  # 초기 위치

        # 첫 메시지와 캐릭터 설명 설정
        self.first_mes = self.process_script(first_mes)
        self.description = self.process_script(description)
        # 캐릭터 현재 상태 초기화
        self.char_goal = self.char_first_goal
        self.char_progress = self.char_first_progress
        self.char_stamina = self.char_first_stamina
        self.char_coordinate = self.char_first_coordinate

        # 연속 공간 관련 설정
        self.obstacles = [(3, 3, 1), (7, 7, 1.5), (5, 5, 1)]  # (x, y, radius)
        self.key_pos = (10, 10)
        self.door_pos = (14, 14)
        self.hybrid_planner = HybridPlanner(self.char_coordinate, self.obstacles)
        self.npc = NPC(self.char_coordinate, self.hybrid_planner)

        # GOAP 액션 정의
        self.actions = [
            Action("MoveToKey", {'has_key': False}, {'npc_pos': self.key_pos}, 1),
            Action("PickUpKey", {'npc_pos': self.key_pos, 'has_key': False}, {'has_key': True}, 1),
            Action("MoveToDoor", {'has_key': True, 'door_open': False}, {'npc_pos': self.door_pos}, 1),
            Action("OpenDoor", {'npc_pos': self.door_pos, 'has_key': True, 'door_open': False}, {'door_open': True}, 1)
        ]

    # 채팅 초기화 메서드
    def initialize_chat(self):
        first_user_message = f"안녕하십니까! 저는 {self.name}님을 호위할 병사 {self.user_name}입니다!"
        self.add_message("user", first_user_message)
        self.add_message("assistant", self.process_script(self.first_mes))

    # 메시지 추가 메서드
    def add_message(self, role: str, content: str):
        self.conversation_history.append({"role": role, "content": content})
        print(f"{'You' if role == 'user' else self.name}: {content}\n")
        self.manage_context_size()

    # 세계 상황 업데이트 메서드
    def update_world_situation(self):
        print("[현재 세계 상황]")
        for key, value in self.world_situation.items():
            print(f"{key}: {value}")
        self.world_situation["Coordinate"] = f"{self.char_coordinate[0]:.2f}, {self.char_coordinate[1]:.2f}"

    # 컨텍스트 크기 관리 메서드
    def manage_context_size(self):
        while sum(len(msg['content']) for msg in self.conversation_history) > self.max_context_tokens and len(self.conversation_history) > 1:
            self.conversation_history.pop(0)

    # API에 메시지 전송 메서드
    def send_message_to_api(self, user_message: str):
        self.update_world_situation()
        request_data = self.prepare_api_request(user_message)

        print("\n--- API 요청 내용 ---")
        print(json.dumps(request_data, indent=2, ensure_ascii=False))
        print("----------------------\n")

        response = self.make_api_request(request_data)

        if response["success"]:
            self.process_api_response(response["content"])
        else:
            print(f"Error: {response['content']}")

    # API 요청 준비 메서드
    def prepare_api_request(self, user_message: str) -> Dict:
        user_status = self.user_status_prompt.format(dialogue=user_message)
        self.add_message("user", user_status)

        world_status = self.format_world_status()
        char_status = self.char_status_prompt.format(goal=self.char_goal, progress=self.char_progress, stamina=self.char_stamina)

        api_conversation_history = copy.deepcopy(self.conversation_history)
        api_conversation_history[-1]['content'] = (
            world_status + "\n" +
            user_status + "\n" +
            char_status + "\n" +
            post_history_instructions
        )

        system_prompt = self.build_system_prompt()

        return {
            "messages": [{"role": "system", "content": system_prompt}] + api_conversation_history
        }

    # API 요청 실행 메서드
    def make_api_request(self, request_data: Dict) -> Dict:
        return self.claude.get_response(request_data)

    # API 응답 처리 메서드
    def process_api_response(self, api_response: str):
        self.add_message("assistant", self.process_script(api_response))

        # GOAP 액션 추출
        actions = re.findall(r"(MoveToKey|PickUpKey|MoveToDoor|OpenDoor)", api_response)

        if actions:
            goap_result = self.execute_goap_actions(actions)
            if goap_result:
                old_goal = self.char_goal
                self.update_character_status(goap_result)
                called_goap = self.prepare_goap_response(old_goal, goap_result)
                self.send_goap_response(api_response, called_goap)

        return True

    def execute_goap_actions(self, actions: List[str]):
        optimized_path = []
        for action_name in actions:
            action = next((a for a in self.actions if a.name == action_name), None)
            if action:
                if action_name in ["MoveToKey", "MoveToDoor"]:
                    target = self.key_pos if action_name == "MoveToKey" else self.door_pos
                    path = self.npc.move_to(target)
                    if path:
                        optimized_path.extend(path)
                        self.char_stamina -= len(path)
                        self.char_coordinate = (path[-1].x, path[-1].y)
                elif action_name == "PickUpKey":
                    self.npc.pick_up_key()
                    self.char_stamina -= 1
                elif action_name == "OpenDoor":
                    self.npc.open_door()
                    self.char_stamina -= 1

                print(f"Executed action: {action_name}")
                print(f"New position: {self.char_coordinate}")
                print(f"Remaining stamina: {self.char_stamina}")

        self.char_progress = "100%"

        # Plot RRT tree and optimized path simultaneously
        self.hybrid_planner.plot_result(optimized_path, self.key_pos, self.door_pos)

        # GOAP 실행 결과 반환
        return actions, self.char_coordinate, self.char_stamina

    # 캐릭터 상태 업데이트 메서드
    def update_character_status(self, goap_result):
        actions, new_position, new_stamina = goap_result
        self.char_coordinate = new_position
        self.char_stamina = new_stamina
        self.char_progress = "100%"

    # GOAP 응답 준비 메서드
    def prepare_goap_response(self, old_goal, goap_result):
        actions, new_position, new_stamina = goap_result
        plan_str = "\n".join([f"- {action}" for action in actions])

        plan_description = f"""{{char}} successfully moved from ({old_goal[0]:.2f}, {old_goal[1]:.2f}) to ({new_position[0]:.2f}, {new_position[1]:.2f}).
Plan executed:
{plan_str}
Remaining stamina: {new_stamina}"""
        return self.process_script(called_goap_prompt.format(feasible="Possible", plan=plan_description))

    # GOAP 응답 전송 메서드
    def send_goap_response(self, api_response, called_goap):
        request_data = self.prepare_api_request(called_goap)
        request_data["messages"].insert(-1, {"role": "assistant", "content": api_response})

        print("\n--- API 요청 내용 (GOAP) ---")
        print(json.dumps(request_data, indent=2, ensure_ascii=False))
        print("------------------------------\n")

        response = self.make_api_request(request_data)

        if response["success"]:
            self.add_message("assistant", self.process_script(response["content"]))
        else:
            print(f"Error: {response['content']}")

    # 세계 상태 포맷팅 메서드
    def format_world_status(self) -> str:
        return self.world_status_prompt.format(
            time=self.world_situation["Time"],
            location=self.world_situation["Location"],
            environment=self.world_situation["Environment"],
            atmosphere=self.world_situation["Atmosphere"],
            surround_cond=self.world_situation["Surrounding Condition"],
            obstacles=', '.join([f"({x:.2f}, {y:.2f}, radius={r:.2f})" for x, y, r in self.obstacles]),
            key_pos=f"({self.key_pos[0]:.2f}, {self.key_pos[1]:.2f})",
            door_pos=f"({self.door_pos[0]:.2f}, {self.door_pos[1]:.2f})",
            coordinate=f"{self.char_coordinate[0]:.2f}, {self.char_coordinate[1]:.2f}"
        )

    # 시스템 프롬프트 구성 메서드
    def build_system_prompt(self):
        return self.process_script(start_prompt.replace("{{char}}", self.name)
                                   .replace("{{user}}", self.user_name).format(description=self.process_script(self.description)))

    # 스크립트 처리 메서드 (플레이스홀더 교체)
    def process_script(self, input_text: str) -> str:
        return input_text.replace("{{char}}", self.name).replace("{{user}}", self.user_name)

In [ ]:
def main():
    ai_manager = RisuAIManager()
    ai_manager.initialize_chat()

    while True:
        print(f"\n캐릭터 목표: {ai_manager.char_goal}")
        print(f"캐릭터 위치: {ai_manager.char_coordinate}")
        print(f"캐릭터 스태미나: {ai_manager.char_stamina}")
        user_input = input("나의 대사: ")
        if user_input.lower() in ['quit', 'exit', '종료']:
            break
        ai_manager.send_message_to_api(user_input)

if __name__ == "__main__":
    main()
